# Maps from Google Cloud ERA5



```{image} ../thumbnails/ecmwf.png
:alt: ECMWF logo
:width: 200px
```

```{image} ../thumbnails/google_cloud.jpg
:alt: Google Cloud logo
:width: 200px
```

---

## Overview
This notebook will go through the process of reading in ERA5 reanalysis data from the Google Cloud archive, and plot an interactive plot with 500-hPa vorticity as an example. This notebook goes over:

1. Reading in ERA5 reanalysis from Google Cloud
1. Subsetting the data to the spatial and temporal coverages of interest without triggering an eager load of the entire ERA5 dataset
1. Analyzing the data and using the utilities of `MetPy` to calculate vorticity
1. Plotting an interactive map with a time slider using `Geoviews` and `Cartopy`
   

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Xarray](https://docs.xarray.dev/en/stable/) | Necessary | Opening and manipulating datasets, lazy loading vs eager loading|
| [Understanding of ERA5](https://www.ecmwf.int/en/forecasts/dataset/ecmwf-reanalysis-v5) | Necessary | Structure of ERA5 |
| Metpy | Helpful | |
| Geoviews | Helpful | |

- **Time to learn**: 30 minutes
- **System requirements**:
    - >20 GB of system memory (Original load of ERA5 data requires some memory)

---

## Imports
--------------------------------

In [38]:
import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime as dt
import metpy.calc as mpcalc
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import gcsfs
import geoviews as gv
import holoviews as hv
from holoviews import opts
import hvplot.xarray
import panel as pn
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.cm as mplcm
from matplotlib.animation import ArtistAnimation
import cartopy.crs as ccrs
import cartopy.feature as cfeat
from metpy.plots import USCOUNTIES, ctables



gv.extension('bokeh')
pn.extension()

## Overview of the Google Cloud ERA5 archive

Google hosts the entirety of the ERA5 reanalysis on its cloud storage system and has made the data publicly accessible from the cloud. The data is organized into analysis ready cloud optimized (ARCO) *Zarr* files which are loacted in so called "buckets", which can be remotely accessed with the help of the `gcsfs` module, which provides a roadmap of the Google Cloud System file storage infrastructure. Zarr formatted files allow for the collection for large groups of data, and are a massive improvement on the previous storage methods which typically store each day's files as separate files. If you wanted to read in data for a long period of time, you would have to deal with lots of files! Zarr allows easy subsetting of the main file so you can pull the data of interest to you.

[**You can browse the Google Cloud hosted ERA5 reanalysis here**](https://console.cloud.google.com/storage/browser/gcp-public-data-arco-era5 'View the Google Cloud ERA5 reanalysis')

## Read in ERA5 from Google Cloud
This uses Google Cloud's python package `gcsfs` to point to the repository for ERA5. `Xarray`'s `xr.open_zarr` is used to remotely open the data from the Google Cloud bucket.

Since the Google Cloud repository of ERA5 data is a public dataset, no credentials are needed. Here, we use the public ananomous token to access the data.

Chunking instructs Xarray's background loader `Dask` on how to open the data. All data is 'lazy loaded', and will be read into memory as it is used in a calculation.

In [12]:
# Use anonymous access (public dataset)
fs = gcsfs.GCSFileSystem(token="anon")

#paths to hourly full chunk era5 data
zarr_path = "gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3"

data = xr.open_zarr(
    fs.get_mapper(zarr_path),
    chunks={"time": 1, "lat": 25, "lon": 25},  # adjust chunking as suited
)
data

<xarray.Dataset> Size: 4PB
Dimensions:                                                          (
                                                                      time: 1323648,
                                                                      latitude: 721,
                                                                      longitude: 1440,
                                                                      level: 37)
Coordinates:
  * time                                                             (time) datetime64[ns] 11MB ...
  * latitude                                                         (latitude) float32 3kB ...
  * longitude                                                        (longitude) float32 6kB ...
  * level                                                            (level) int64 296B ...
Data variables: (12/273)
    100m_u_component_of_wind                                         (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    100m_v_component_of_wind                                         (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    10m_u_component_of_neutral_wind                                  (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    10m_u_component_of_wind                                          (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    10m_v_component_of_neutral_wind                                  (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    10m_v_component_of_wind                                          (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    ...                                                               ...
    wave_spectral_directional_width_for_swell                        (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    wave_spectral_directional_width_for_wind_waves                   (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    wave_spectral_kurtosis                                           (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    wave_spectral_peakedness                                         (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    wave_spectral_skewness                                           (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    zero_degree_level                                                (time, latitude, longitude) float32 5TB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
Attributes:
    last_updated:           2025-11-26 02:14:09.972189+00:00
    valid_time_start:       1940-01-01
    valid_time_stop:        2025-04-30
    valid_time_stop_era5t:  2025-11-20

In [13]:
print (f'The entire zarr file is {data.nbytes/1e12:.04g} terabytes big!')

The entire zarr file is 3678 terabytes big!


## Select Range of Dates to Encompass Your Event of Interest
This uses `pd.date_range` to create a list of times on some interval from your start to end time. We will need to adjust the formatting of the dates to be that of the time stamps used in ERA5: YYYY-MM-DD-HH:MM:SS.000000000

To select the 4 May 2018 Merrimack County NH EF1, set `ex_opt` to 0
To select the 29 July 2021 Central New Jersey outbreak, set `ex_opt` to 1

If you want to visualize a different event, change the dates below to the event of your choice

In [65]:
#=======Select what case to view============
ex_opt=0
#===========================================

if ex_opt==0:
    #dates for the 2018 Merrimack County tornado
    startYear = 2018
    startMonth = 5
    startDay = 3
    startHour = 0
    startTime = dt(startYear,startMonth,startDay,startHour)
    
    endYear = 2018
    endMonth = 5
    endDay = 6
    endHour = 0
    endTime = dt(endYear,endMonth,endDay,endHour)
    
    dateList = pd.date_range(startTime, endTime,freq="6h")
    
elif ex_opt==1:
    startYear = 2021
    startMonth = 7
    startDay = 28
    startHour = 0
    startTime = dt(startYear,startMonth,startDay,startHour)
    
    endYear = 2021
    endMonth = 7
    endDay = 31
    endHour = 0
    endTime = dt(endYear,endMonth,endDay,endHour)
    
    dateList = pd.date_range(startTime, endTime,freq="6h")


#put date list into format used for times in ERA5
dates = [dt.strftime('%Y-%m-%dT%H:%M:%S') + '.000000000' for dt in dateList]
dates

['2018-05-03T00:00:00.000000000',
 '2018-05-03T06:00:00.000000000',
 '2018-05-03T12:00:00.000000000',
 '2018-05-03T18:00:00.000000000',
 '2018-05-04T00:00:00.000000000',
 '2018-05-04T06:00:00.000000000',
 '2018-05-04T12:00:00.000000000',
 '2018-05-04T18:00:00.000000000',
 '2018-05-05T00:00:00.000000000',
 '2018-05-05T06:00:00.000000000',
 '2018-05-05T12:00:00.000000000',
 '2018-05-05T18:00:00.000000000',
 '2018-05-06T00:00:00.000000000']

## Select your region of interest
**NOTE**: ERA5 longitude is given in degrees east from 0-360. For locations in the Western hemisphere, express longitude as a negative value (i.e.-75), then add 360 to it to account for the ERA5 coordinate system

Here we will grab data for the eastern half of CONUS

In [66]:
latS=20
latN=55
lonW=-110+360
lonE=-50+360

## Subset the data to the times, region, and variables of interest
This takes advantage of xarray's `xr.sel` function to process the zarr file of the entire reanalysis into what you want
**NOTE** Using slice() prevents an eager load of the entire dataset, which is around 4 PB of data!

Since we are interested in plotting vorticity, we will only need to keep 3 variables out of the 273 total: u, v, and geopotential

Once we have subsetted the data, force a load of the data, which is now much more manageable. This load might take a bit of time now, but can greatly speed up future processes if they have to iterate over time.

Lastly, select and subset the pressure level needed, in this case 500 hPa

In [67]:
#only load what variables are needed
vars_to_keep=['u_component_of_wind','v_component_of_wind','geopotential']

print('subsetting data')
ds1h=data[vars_to_keep].sel(latitude=slice(latN,latS),longitude=slice(lonW,lonE),time=slice(dates[0], dates[-1]))

#the data is hourly but we only want 6 hour synoptic time intervals. Pull out every 6th hour of data to further reduce size
ds=ds1h.isel(time=slice(0, None, 6))

print(f'The subsetted is {ds.nbytes/1e6:.04g} megabytes')
#force the data to load to memory
ds.load()

# subset the pressure level
plevel=500
ds=ds.sel(level=plevel)

subsetting data
The subsetted is 196.1 megabytes


## Define variables of interest
Note that we want to plot geopotential height, however ERA5 uses geopotential as its height coordinate. We will have to convert to geopotential height later on.

In [68]:
u=ds.u_component_of_wind
v=ds.v_component_of_wind
g=ds.geopotential
lats=ds.latitude.values
lons=ds.longitude.values

## Convert Geopotential to Geopotential Height
ERA5 uses geopotential as its height coordinate, however it is standard to plot contours of geopotential height. Using MetPy's `mpcalc.geopotential_to_height` we easily convert. 
Since we are looking to plot multiple times, convert to geopotential height for each time step

MetPy's `metpy.quantify` will parse the data to pull out the pre assigned units, so we do not have to go through the hassle of attaching units to everything

In [69]:
# function for one timestep only
def gh_one_timestep(g):
    z_m = mpcalc.geopotential_to_height(g.metpy.quantify())
    return z_m

# apply across all times
gh_list = []
for t in dates:
    gh_list.append(gh_one_timestep(g.sel(time=t)))

z_m = xr.concat(gh_list, dim='time')
z_m = z_m.assign_coords(time=dates)
z_m.attrs['units'] = 'm'
z_m = z_m.assign_coords(time=pd.to_datetime(z_m.time.values)) 

# convert to decameters, which is the plotting standard. 1 dam = 10 meters
z_dam=z_m/10


## Use MetPy's `mpcalc.vorticity` to calculate vorticity, and scale it to a nicer magnitude
Metpy will also need to know the grid spacing to accurately calculate vorticity. Calculate dx and dy and feed them into the calculation of vorticity. Feed latitude into the calculation as a 2-d grid as well.

Repeat the same per-time calculation process as above

Again, `metpy.quantify` allows avoidance of having to attach and deal with units


In [70]:
# calculate grid spacing
dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)

#create 2-d grid of lat/lon for mpcalc.vorticity to use to determine coriolis parameter
lon2d, lat2d = np.meshgrid(lons, lats)

# function for one timestep only
def vort_one_timestep(u2d, v2d):
    vort = mpcalc.vorticity(
        u2d.metpy.quantify(),
        v2d.metpy.quantify(),
        dx=dx,
        dy=dy,
        latitude=np.radians(lat2d)
    )
    return vort * 1e5  # scale to 10^-5 s^-1

# apply across all times
vorticity_list = []
for t in dates:
    vorticity_list.append(vort_one_timestep(u.sel(time=t), v.sel(time=t)))

vorticity = xr.concat(vorticity_list, dim='time')
vorticity = vorticity.assign_coords(time=u.time)
vorticity.attrs['units'] = '1e-5 s^-1'


## Animate 500-hPa vorticiy with MatPlotLib's animation function

In [ ]:
# setup map projection. For this example, we setup a center of the plot to cover the Northeast as a whole
cLat = 41.0                   # central latitude of the projection
cLon = -77.5                   # central longitude of the projection
    # create the projection and an axis object
proj=ccrs.LambertConformal(central_latitude=cLat, central_longitude=cLon)

#settings for animation
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 100
plt.ioff()

#determine the number of time steps
ntimes=np.shape(vorticity)[0]

#declare the figure item
figanim=plt.figure(figsize=(12,12))
ax = figanim.add_subplot(111, projection=proj)

#setup a colorbar, need to create dummy scalar mappable to plot colorbar
norm = plt.Normalize(vmin=0, vmax=50)
cmap = 'plasma_r'

sm = mplcm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])

# add colorbar to figure
cbar = figanim.colorbar(sm, ax=ax, orientation='vertical', shrink=0.25, pad=0.02)
cbar.set_label('500-hPa Relative Vorticity [10-5 s-1]')




def animate(t):
    ax.cla()

    mtime=np.copy(t)
    print(f'Frame {mtime}/{ntimes-1}',end='\r')

    #subset to current time step
    vort_sel=vorticity.isel(time=mtime)
    u_sel=u.isel(time=mtime)
    v_sel=v.isel(time=mtime)
    z_dam_sel=z_dam.isel(time=mtime)

    #pull current time string
    current_time=vort_sel.time.values
    print(current_time)

    
    
    # add some nice map features, zoom in on area of interest
    ax.set_extent([-100,-60,50,30])
    
    ax.add_feature(cfeat.LAND.with_scale('10m'),color='gray')
    ax.add_feature(cfeat.STATES.with_scale('10m'),edgecolor='black', linewidth=1.5,zorder=6)
    #ax.add_feature(USCOUNTIES.with_scale('5m'),edgecolor='w', linewidth=1.5,zorder=6 )

    
    #plot 500-hPa vort
    levs=np.arange(0,50,2)
    cm = ax.contourf(vort_sel.longitude.values,vort_sel.latitude.values,vort_sel.values,levels=levs,cmap=cmap,transform=ccrs.PlateCarree(),zorder=5)

    #plot geopotential height contours
    CL = ax.contour(vort_sel.longitude.values,vort_sel.latitude.values,z_dam_sel.values,levels=np.arange(420,630,6),linewidths=2.5,transform=ccrs.PlateCarree(),colors='black',zorder=8)
    ax.clabel(CL, inline_spacing=0.2, fontsize=18, fmt='%.0f')

    
    #plot wind barbs
    ax.barbs(x=vort_sel.longitude[::20].values, y=vort_sel.latitude[::20].values,
             u=u_sel[::20,::20].values, v=v_sel[::20,::20].values, length=8, barbcolor='black', flagcolor='red',transform=ccrs.PlateCarree(),zorder=7)

    time=dt.strptime(str(current_time)[:16], "%Y-%m-%dT%H:%M")
    timeStr=time.strftime('%Y-%m-%d %H:%Mz')
    title=f'500-hPa Relative Vorticity [10-5 s-1], Geopotential Height [dam] \n Valid: {timeStr}'
    plt.title(title)
    

anim=matplotlib.animation.FuncAnimation(figanim, animate, frames=ntimes)
anim
                  


2018-05-03T00:00:00.000000000
2018-05-03T00:00:00.000000000
2018-05-03T06:00:00.000000000
2018-05-03T12:00:00.000000000
2018-05-03T18:00:00.000000000
2018-05-04T00:00:00.000000000
2018-05-04T06:00:00.000000000
2018-05-04T12:00:00.000000000
2018-05-04T18:00:00.000000000
2018-05-05T00:00:00.000000000
2018-05-05T06:00:00.000000000


-------------------------------------

## Summary
We were able to access ARCO formatted ERA5 reanalysis from Google Cloud, subset the large dataset to one select gropu of times and region of interest, reducing the size of the dataset to one that most systems can easily handle. We then were able to work with the data to calculate vortitcity, and plot it using a `GeoViews` interactive plot with a time slider to easily parse multiple dates.

### Other things to try
- Try plotting other fields. ERA5 contains a bunch of pre-calculated fields such as CAPE, vorticity, etc.
- Change the domain or time subset to plot an event that interests you
### What's next?
The next notebook will go over plotting modeled soundings from ERA5 using `SounderPy`.

## Resources and references
 - ERA5 on Google Cloud: https://github.com/google-research/arco-era5
 - Micheal Barletta's ERA5 Interactive Cookbook: https://github.com/Michael-Barletta/ERA5_interactive-cookbook
 - [Holoviews/Geoviews](https://hvplot.holoviz.org/en/docs/latest/index.html)
 
